In [1]:
#import os as os
#os.system('pip install business_duration')
#os.system('pip install holidays')
import numpy as np
from business_duration import businessDuration
import pandas as pd
from datetime import time,datetime
import holidays as pyholidays
from datetime import timedelta

In [19]:
df = pd.read_excel('Phase2DataFile(1).xlsx')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163768 entries, 0 to 163767
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Rate/hr                 163768 non-null  float64       
 1   All Day Rate            163768 non-null  int64         
 2   Minutes for Daily Rate  163768 non-null  int64         
 3   Min Minutes             163768 non-null  int64         
 4   Min Minutes rate        163768 non-null  float64       
 5   LotNumber               163768 non-null  object        
 6   Lot Address             163768 non-null  object        
 7   Days of Operation       163768 non-null  object        
 8   Beginning Hour          163768 non-null  object        
 9   Ending Hour             163768 non-null  object        
 10  Vehicle                 163768 non-null  object        
 11  Entry                   163768 non-null  datetime64[ns]
 12  Entry Date              163768

In [20]:
df['Close'] = pd.to_datetime(df['Exit Day'].astype(str)+ ' ' +df['Ending Hour'].astype(str))
df['Open'] = pd.to_datetime(df['Entry Date'].astype(str)+ ' ' +df['Beginning Hour'].astype(str))


In [33]:
closes_am = df['Ending Hour'] < df['Beginning Hour']
df["End"]=pd.to_datetime(df['Exit'])
df["Start"]=pd.to_datetime(df['Entry'])
df.loc[closes_am & (df['End'] < df['Start']), 'End'] = df['End'] + timedelta(days = 1, hours = 0, minutes = 0, seconds = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163768 entries, 0 to 163767
Data columns (total 37 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Rate/hr                 163768 non-null  float64       
 1   All Day Rate            163768 non-null  int64         
 2   Minutes for Daily Rate  163768 non-null  int64         
 3   Min Minutes             163768 non-null  int64         
 4   Min Minutes rate        163768 non-null  float64       
 5   LotNumber               163768 non-null  object        
 6   Lot Address             163768 non-null  object        
 7   Days of Operation       163768 non-null  object        
 8   Beginning Hour          163768 non-null  object        
 9   Ending Hour             163768 non-null  object        
 10  Vehicle                 163768 non-null  object        
 11  Entry                   163768 non-null  datetime64[ns]
 12  Entry Date              163768

In [34]:
df.loc[closes_am, 'Start'] = df['Start'] - timedelta(days = 0, hours = 3, minutes = 0, seconds = 0)
df.loc[closes_am, 'End'] = df['End'] - timedelta(days = 0, hours = 3, minutes = 0, seconds = 0)
#shift back all the entry and exit hours by 3 hours, to shift back this day from 2AM-->11PM and from 9AM-->6AM or 8AM-->5AM.
#It should be noted that the opening hour and the closing hour need to shift back as well that is done in the next few lines

In [35]:
df['close_hour'] = df.Close.dt.hour
df['open_hour'] = df.Open.dt.hour
df.loc[closes_am, 'close_hour'] = 23  #**
#All lots that close at 2AM will now be closed at a shifted 11PM, so set the close hour for 23 hundred hours
df.loc[closes_am, 'open_hour'] = df['open_hour'] -3
#Shift back the opening hour by 3 hours


In [36]:
weekendcondition= df['Days of Operation']=='Monday-Saturday'
df['Weekends']=None
df.loc[weekendcondition,'Weekends']=6
#This is to take into account weekends, it automatically charges 0 chargeable minutes on Sundays but only if the row meets
#weekendcondition
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163768 entries, 0 to 163767
Data columns (total 37 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Rate/hr                 163768 non-null  float64       
 1   All Day Rate            163768 non-null  int64         
 2   Minutes for Daily Rate  163768 non-null  int64         
 3   Min Minutes             163768 non-null  int64         
 4   Min Minutes rate        163768 non-null  float64       
 5   LotNumber               163768 non-null  object        
 6   Lot Address             163768 non-null  object        
 7   Days of Operation       163768 non-null  object        
 8   Beginning Hour          163768 non-null  object        
 9   Ending Hour             163768 non-null  object        
 10  Vehicle                 163768 non-null  object        
 11  Entry                   163768 non-null  datetime64[ns]
 12  Entry Date              163768

In [37]:
holidaylist = pyholidays.Canada()
#This takes into account Canadian holidays

df['chargeable_mins'] = df.apply(lambda x: businessDuration(startdate = x['Start'],enddate = x['End'], \
                                                               starttime =time(x['open_hour'], 0,0),\
                                                               endtime= time(x['close_hour'], 0,0), \
                                                               holidaylist= holidaylist,\
                                                               weekendlist = [x['Weekends']],\
                                                               unit='min'), axis =1)
#We calculate the chargeable minutes

In [38]:
df['HoursOpen']=(df['close_hour']-df['open_hour'])*60
#a full day is no longer 1440 mins, its the number of hours open *60
df['NumDays']=np.floor(df['chargeable_mins']/df['HoursOpen'])
df['RemainingMins']=np.ceil(np.mod(df['chargeable_mins'], df['HoursOpen']))

condition1= (df['chargeable_mins']) == 0
# then 0
condition2 = (df['chargeable_mins'] >= df['Minutes for Daily Rate']) & (df['RemainingMins'] == 0)
#then daily rate*df['Numdays']
condition3 = (df['chargeable_mins'] >= df['Minutes for Daily Rate']) & (df['RemainingMins'] >= df['Minutes for Daily Rate']) 
#then rate*df['Numdays'] + daily rate
condition4 = (df['chargeable_mins'] >= df['Minutes for Daily Rate']) & (df['RemainingMins'] < df['Minutes for Daily Rate']) 
#then rate*df['Numdays'] + df['RemainingMins']* Rate/minmin
condition5= (df['chargeable_mins'])<(df['Minutes for Daily Rate'])
#then chargemins*Rate/minmin
condition6= (df['chargeable_mins']>0) & (df['Minutes for Daily Rate']==0)
#then chargemins*Rate/minmin


In [39]:
df.loc[condition1, 'PlotFee'] =0
df.loc[condition2, 'PlotFee'] = df['NumDays']*df['All Day Rate']
df.loc[condition3, 'PlotFee'] = (df['NumDays']+1)*df['All Day Rate']
df.loc[condition4, 'PlotFee'] = df['NumDays']*df['All Day Rate'] + np.ceil(df['RemainingMins']/df['Min Minutes'])*df['Min Minutes rate']
df.loc[condition5, 'PlotFee'] = np.ceil(df['chargeable_mins']/df['Min Minutes'])*df['Min Minutes rate']
df.loc[condition6, 'PlotFee'] = np.ceil(df['chargeable_mins']/df['Min Minutes'])*df['Min Minutes rate']
#calculate the fees


In [40]:
df.loc[df['chargeable_mins']>0, ['End','Start','PlotFee','chargeable_mins','Min Minutes rate',\
                           'All Day Rate','NumDays','RemainingMins','Min Minutes','Minutes for Daily Rate']]
#this line is for sanity checks, the text before the first comma '~weekends' can be replaced with anything 
#like df['PlotFee']== 4, that way if we see something wrong we can try to debug

,End,Start,PlotFee,chargeable_mins,Min Minutes rate,All Day Rate,NumDays,RemainingMins,Min Minutes,Minutes for Daily Rate
0,2021-09-01 20:00:00,2021-09-01 03:00:00,12.00,840.000000,1.00,12,0.0,840.0,20,240
1,2021-09-01 20:00:00,2021-09-01 03:00:00,12.00,840.000000,1.00,12,0.0,840.0,20,240
2,2021-09-01 20:00:00,2021-09-01 03:00:00,12.00,840.000000,1.00,12,0.0,840.0,20,240
3,2021-09-01 20:00:00,2021-09-01 03:00:00,12.00,840.000000,1.00,12,0.0,840.0,20,240
4,2021-09-01 20:00:00,2021-09-01 03:00:00,12.00,840.000000,1.00,12,0.0,840.0,20,240
...,...,...,...,...,...,...,...,...,...,...
163763,2019-10-31 23:00:01,2019-10-31 14:33:01,12.00,506.983333,1.00,12,0.0,507.0,20,240
163764,2019-10-31 17:56:44,2019-10-31 15:16:44,4.00,160.000000,1.00,10,0.0,160.0,40,400
163765,2019-10-31 13:36:33,2019-10-31 13:16:33,0.25,20.000000,0.25,5,0.0,20.0,20,400
163766,2019-10-31 09:40:34,2019-10-31 08:20:34,4.00,80.000000,1.00,12,0.0,80.0,20,240


In [42]:
df.to_excel("Phase2DataFees.xlsx")